In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [4]:
location = "datasets/Kickstartercombo.csv"

df = pd.read_csv(location)
df.head()

,Unnamed: 0,ID,name,main_category,category,country,state,goal,pledged,currency,launched,deadline,backers,usd pledged,usd_goal_real,usd_pledged_real
0,0,1000002330,The Songs of Adelaide & Abullah,Publishing,Poetry,GB,failed,1000.0,0.0,GBP,2015-08-11 12:12:00,2015-10-09 11:36:00,0,0.0,NaN,NaN
1,1,1000004038,Where is Hank?,Film & Video,Narrative Film,US,failed,45000.0,220.0,USD,2013-01-12 00:20:00,2013-02-26 00:20:00,3,220.0,NaN,NaN
2,2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,US,failed,5000.0,1.0,USD,2012-03-17 03:24:00,2012-04-16 04:24:00,1,1.0,NaN,NaN
3,3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,US,canceled,19500.0,1283.0,USD,2015-07-04 08:35:00,2015-08-29 01:00:00,14,1283.0,NaN,NaN
4,4,1000014025,Monarch Espresso Bar,Food,Restaurants,US,successful,50000.0,52375.0,USD,2016-02-26 13:38:00,2016-04-01 13:38:00,224,52375.0,NaN,NaN


In [5]:
df.shape

(702411, 16)

Clean the Data

In [6]:
df.describe()

,Unnamed: 0,ID,goal,pledged,backers,usd pledged,usd_goal_real,usd_pledged_real
count,702411.000000,7.024110e+05,7.024110e+05,7.024110e+05,702411.000000,6.948170e+05,3.786610e+05,3.786610e+05
mean,351205.000000,1.074829e+09,4.817642e+04,9.223828e+03,103.726589,7.405969e+03,4.545440e+04,9.058924e+03
std,202768.734302,6.192006e+08,1.162920e+06,9.286016e+04,919.654297,8.144094e+04,1.152950e+06,9.097334e+04
min,0.000000,5.971000e+03,1.000000e-02,0.000000e+00,0.000000,0.000000e+00,1.000000e-02,0.000000e+00
25%,175602.500000,5.379594e+08,2.000000e+03,3.000000e+01,2.000000,2.024000e+01,2.000000e+03,3.100000e+01
50%,351205.000000,1.075562e+09,5.000000e+03,6.150000e+02,12.000000,4.560000e+02,5.500000e+03,6.243300e+02
75%,526807.500000,1.610544e+09,1.505600e+04,4.020000e+03,55.000000,3.261000e+03,1.550000e+04,4.050000e+03
max,702410.000000,2.147476e+09,1.000000e+08,2.033899e+07,219382.000000,2.033899e+07,1.663614e+08,2.033899e+07


In [12]:
test= df.isnull().sum()
test

Unnamed: 0               0
ID                       0
name                     8
main_category            0
category                 0
country                  0
state                    0
goal                     0
pledged                  0
currency                 0
launched                 0
deadline                 0
backers                  0
usd pledged           7594
usd_goal_real       323750
usd_pledged_real    323750
dtype: int64

In [13]:
modeldf = df.drop(['ID','name','usd_goal_real', 'usd_pledged_real'], axis=1)

In [37]:
modeldf.shape

(702411, 12)

In [17]:
modeldf.isnull().sum()

Unnamed: 0          0
main_category       0
category            0
country             0
state               0
goal                0
pledged             0
currency            0
launched            0
deadline            0
backers             0
usd pledged      7594
dtype: int64

In [38]:
#drop rows with any missing data
modeldf_no_missing = modeldf.dropna()
modeldf_no_missing

,Unnamed: 0,main_category,category,country,state,goal,pledged,currency,launched,deadline,backers,usd pledged
0,0,Publishing,Poetry,GB,failed,1000.0,0.00,GBP,2015-08-11 12:12:00,2015-10-09 11:36:00,0,0.000000
1,1,Film & Video,Narrative Film,US,failed,45000.0,220.00,USD,2013-01-12 00:20:00,2013-02-26 00:20:00,3,220.000000
2,2,Music,Music,US,failed,5000.0,1.00,USD,2012-03-17 03:24:00,2012-04-16 04:24:00,1,1.000000
3,3,Film & Video,Film & Video,US,canceled,19500.0,1283.00,USD,2015-07-04 08:35:00,2015-08-29 01:00:00,14,1283.000000
4,4,Food,Restaurants,US,successful,50000.0,52375.00,USD,2016-02-26 13:38:00,2016-04-01 13:38:00,224,52375.000000
5,5,Food,Food,US,successful,1000.0,1205.00,USD,2014-12-01 18:30:00,2014-12-21 18:30:00,16,1205.000000
6,6,Food,Drinks,US,failed,25000.0,453.00,USD,2016-02-01 20:05:00,2016-03-17 19:05:00,40,453.000000
7,7,Design,Product Design,US,canceled,125000.0,8233.00,USD,2014-04-24 18:14:00,2014-05-29 18:14:00,58,8233.000000
8,8,Film & Video,Documentary,US,canceled,65000.0,6240.57,USD,2014-07-11 21:55:00,2014-08-10 21:55:00,43,6240.570000
9,9,Publishing,Nonfiction,CA,failed,2500.0,0.00,CAD,2013-09-09 18:19:00,2013-10-09 18:19:00,0,0.000000


In [40]:
#find columns that have missing values
modeldf_no_missing.isnull().sum()

Unnamed: 0       0
main_category    0
category         0
country          0
state            0
goal             0
pledged          0
currency         0
launched         0
deadline         0
backers          0
usd pledged      0
dtype: int64

In [41]:
#creates a table of correlation values
modeldf_no_missing.corr()

,Unnamed: 0,goal,pledged,backers,usd pledged
Unnamed: 0,1.000000,0.002168,0.004830,0.001820,-0.003919
goal,0.002168,1.000000,0.007088,0.004405,0.005871
pledged,0.004830,0.007088,1.000000,0.729665,0.896962
backers,0.001820,0.004405,0.729665,1.000000,0.705763
usd pledged,-0.003919,0.005871,0.896962,0.705763,1.000000


In [42]:
#dummy variables for state of kickstarter funding
#get_dummies will auto-drop columns that dummies were created from
modeldf_status = pd.get_dummies(data=modeldf_no_missing, columns=['state'])
modeldf_status.head()

,Unnamed: 0,main_category,category,country,goal,pledged,currency,launched,deadline,backers,usd pledged,state_canceled,state_failed,state_live,state_successful,state_suspended
0,0,Publishing,Poetry,GB,1000.0,0.0,GBP,2015-08-11 12:12:00,2015-10-09 11:36:00,0,0.0,0,1,0,0,0
1,1,Film & Video,Narrative Film,US,45000.0,220.0,USD,2013-01-12 00:20:00,2013-02-26 00:20:00,3,220.0,0,1,0,0,0
2,2,Music,Music,US,5000.0,1.0,USD,2012-03-17 03:24:00,2012-04-16 04:24:00,1,1.0,0,1,0,0,0
3,3,Film & Video,Film & Video,US,19500.0,1283.0,USD,2015-07-04 08:35:00,2015-08-29 01:00:00,14,1283.0,1,0,0,0,0
4,4,Food,Restaurants,US,50000.0,52375.0,USD,2016-02-26 13:38:00,2016-04-01 13:38:00,224,52375.0,0,0,0,1,0


In [30]:
modeldf.isnull().sum()

Unnamed: 0          0
main_category       0
category            0
country             0
state               0
goal                0
pledged             0
currency            0
launched            0
deadline            0
backers             0
usd pledged      7594
dtype: int64

In [43]:
modeldf_status.corr()

,Unnamed: 0,goal,pledged,backers,usd pledged,state_canceled,state_failed,state_live,state_successful,state_suspended
Unnamed: 0,1.000000,0.002168,0.004830,0.001820,-0.003919,0.003458,0.001869,-0.027399,0.001334,0.002338
goal,0.002168,1.000000,0.007088,0.004405,0.005871,0.011087,0.015264,0.001801,-0.024598,0.008831
pledged,0.004830,0.007088,1.000000,0.729665,0.896962,-0.024665,-0.089316,0.001575,0.108304,0.001022
backers,0.001820,0.004405,0.729665,1.000000,0.705763,-0.028946,-0.100760,-0.003255,0.124078,0.000319
usd pledged,-0.003919,0.005871,0.896962,0.705763,1.000000,-0.022642,-0.080412,-0.006416,0.099450,0.000884
state_canceled,0.003458,0.011087,-0.024665,-0.028946,-0.022642,1.000000,-0.356466,-0.034649,-0.250841,-0.023407
state_failed,0.001869,0.015264,-0.089316,-0.100760,-0.080412,-0.356466,1.000000,-0.108267,-0.783791,-0.073139
state_live,-0.027399,0.001801,0.001575,-0.003255,-0.006416,-0.034649,-0.108267,1.000000,-0.076186,-0.007109
state_successful,0.001334,-0.024598,0.108304,0.124078,0.099450,-0.250841,-0.783791,-0.076186,1.000000,-0.051467
state_suspended,0.002338,0.008831,0.001022,0.000319,0.000884,-0.023407,-0.073139,-0.007109,-0.051467,1.000000


In [45]:
modeldf_status.to_csv('cleaned_dataset.csv')